In [3]:
# Break Down & Run the code

from src.feedback.manual import manual_feedback as all_feedback # Load feedback
# Qualitive feedback how to evaluate? 

feedback = all_feedback[-1]


# 1. Custom Trainer Object

# 2. Dataset preparation 

# 3. Modal Training


In [1]:
from src.logger import logger
from src.models import get_model
from src.dataset.feedback_utils import Feedback, Type
from src.lcdpo import LocallyConstrainedDPOTrainer
from src.sft_weighted import WeightedSFTTrainer
from src.dataset.format import to_dpo, to_sft, to_lcdpo, to_sft_weighted
from src.feedback import manual_feedback as all_feedback
from src.utils import get_args, find_all_linear_names, dump_arg_dicts, PeftSavingCallback, get_train_file_name, print_num_trainable_params, TrainingArguments, find_file_with_prefix

'NoneType' object has no attribute 'cadam32bit_grad_fp32'


/Users/fangyuanyu/anaconda3/lib/python3.11/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


In [2]:
WeightedSFTTrainer

src.sft_weighted.WeightedSFTTrainer